<a href="https://colab.research.google.com/github/ParkkaviSivakaran72/Dog-Vs-Cat-classification/blob/main/Lung_Cancer_Patient_Survival_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
#installing the kaggle library
!pip install kaggle

In [55]:
#configuring the path kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [56]:
!kaggle competitions download -c idealize-2025-datathon-competition

idealize-2025-datathon-competition.zip: Skipping, found more recently modified local copy (use --force to force download)


In [57]:
from zipfile import ZipFile

dataset = '/content/idealize-2025-datathon-competition.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall();
  print('Successfully extracting the dataset zip file')


Successfully extracting the dataset zip file


In [58]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [59]:
cancer_data = pd.read_csv('/content/train.csv')

In [60]:
cancer_data.head()

,record_id,first_name,last_name,sex,patient_age,residence_state,height_cm,weight_kg,smoking_status,cigarettes_per_day,...,has_other_cancer,asthma_diagnosis,liver_condition,blood_pressure_status,diagnosis_date,cancer_stage,treatment_start_date,treatment_type,treatment_end_date,survival_status
0,1,Kathryn,Pope,Male,51,New Hampshire,170,48,Passive Smoker,NaN,...,No,Yes,Has Cirrhosis,High Blood Pressure,2022-10-14,4,2022-10-18,Radiation,2023-09-06,0
1,2,Kristin,Nichols,Male,52,Florida,173,59,Never Smoked,NaN,...,No,No,Normal,High Blood Pressure,2018-03-08,4,2018-03-10,Combined,2019-04-07,0
2,3,Oscar,Castillo,Female,62,Washington,163,84,Current Smoker,5.0,...,Yes,No,Normal,High Blood Pressure,2016-03-01,4,2016-03-05,Chemotherapy,2017-06-10,0
3,4,Danielle,Duran,Female,46,Alabama,147,90,Never Smoked,NaN,...,No,No,Normal,Normal,2019-04-18,2,2019-04-24,Surgery,2020-08-11,1
4,5,Shawn,Reed,Male,65,Montana,173,64,Current Smoker,1.0,...,No,Yes,Has Cirrhosis,Normal,2020-05-16,4,2020-05-23,Surgery,2021-04-24,0


In [61]:
selected_features = ['patient_age','smoking_status','cigarettes_per_day','has_other_cancer','asthma_diagnosis','liver_condition	','blood_pressure_status','diagnosis_date','cancer_stage','treatment_start_date','treatment_type','treatment_end_date']


In [62]:
combined_features = (
    cancer_data['patient_age'].astype(str) + ' ' +
    cancer_data['smoking_status'].astype(str) + ' ' +
    cancer_data['cigarettes_per_day'].astype(str) + ' ' +
    cancer_data['has_other_cancer'].astype(str) + ' ' +
    cancer_data['asthma_diagnosis'].astype(str) + ' ' +
    cancer_data['liver_condition'].astype(str) + ' ' +
    cancer_data['blood_pressure_status'].astype(str) + ' ' +
    cancer_data['diagnosis_date'].astype(str) + ' ' +
    cancer_data['cancer_stage'].astype(str) + ' ' +
    cancer_data['treatment_start_date'].astype(str) + ' ' +
    cancer_data['treatment_type'].astype(str) + ' ' +
    cancer_data['treatment_end_date'].astype(str)
)


In [63]:
print(combined_features)

0         51 Passive Smoker nan No Yes Has Cirrhosis Hig...
1         52 Never Smoked nan No No Normal High Blood Pr...
2         62 Current Smoker 5.0 Yes No Normal High Blood...
3         46 Never Smoked nan No No Normal Normal 2019-0...
4         65 Current Smoker 1.0 No Yes Has Cirrhosis Nor...
                                ...                        
999994    55 Current Smoker 2.0 No Yes Has Cirrhosis Hig...
999995    59 Never Smoked nan No Yes Normal High Blood P...
999996    52 Passive Smoker nan No No Normal Normal 2017...
999997    62 Never Smoked nan Yes No Normal High Blood P...
999998    55 Former Smoker 7.0 No Yes Normal High Blood ...
Length: 999999, dtype: object


In [64]:

import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [65]:
vectorizer = TfidfVectorizer()

In [66]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [67]:
# Transform text into sparse matrix
featured_vectors = vectorizer.fit_transform(combined_features)

# Convert to dense array
dense_array = featured_vectors.toarray()

# Get feature names
feature_names = vectorizer.get_feature_names_out()

# Create DataFrame
df_features = pd.DataFrame(dense_array, columns=feature_names)

# View
print(df_features.head())


        01   02        03        04        05        06        07        08  \
0  0.00000  0.0  0.000000  0.000000  0.000000  0.197889  0.000000  0.000000   
1  0.00000  0.0  0.414522  0.208382  0.000000  0.000000  0.206596  0.206730   
2  0.21173  0.0  0.423335  0.000000  0.211006  0.213180  0.000000  0.000000   
3  0.00000  0.0  0.000000  0.397356  0.000000  0.000000  0.000000  0.197104   
4  0.00000  0.0  0.000000  0.193727  0.384165  0.000000  0.000000  0.000000   

         09        10  ...   ok   passive  pressure  radiation  smk    smoked  \
0  0.197547  0.380208  ...  0.0  0.200919  0.108438   0.201205  0.0  0.000000   
1  0.000000  0.200530  ...  0.0  0.000000  0.114385   0.000000  0.0  0.212172   
2  0.000000  0.204793  ...  0.0  0.000000  0.116817   0.000000  0.0  0.000000   
3  0.000000  0.000000  ...  0.0  0.000000  0.000000   0.000000  0.0  0.202292   
4  0.000000  0.000000  ...  0.0  0.000000  0.000000   0.000000  0.0  0.000000   

     smoker  surg   surgery       yes 

In [68]:
from sklearn.model_selection import train_test_split
labels = cancer_data['survival_status']
X_train, X_test, y_train, y_test = train_test_split(
    df_features,
    labels,
    test_size=0.2,
    random_state=42
)


In [69]:
model = LogisticRegression(class_weight='balanced')

In [70]:
print(X_train)

              01        02        03        04        05        06   07   08  \
566853  0.201969  0.206004  0.000000  0.000000  0.000000  0.000000  0.0  0.0   
382311  0.361149  0.184182  0.000000  0.000000  0.000000  0.000000  0.0  0.0   
241519  0.000000  0.184294  0.180631  0.000000  0.000000  0.000000  0.0  0.0   
930120  0.000000  0.000000  0.192763  0.387610  0.192160  0.194139  0.0  0.0   
911069  0.206007  0.000000  0.000000  0.000000  0.000000  0.207417  0.0  0.0   
...          ...       ...       ...       ...       ...       ...  ...  ...   
259178  0.192470  0.000000  0.000000  0.000000  0.383625  0.387576  0.0  0.0   
365838  0.000000  0.000000  0.392433  0.000000  0.000000  0.000000  0.0  0.0   
131932  0.229882  0.234475  0.000000  0.000000  0.000000  0.000000  0.0  0.0   
671155  0.177068  0.000000  0.354032  0.000000  0.000000  0.178280  0.0  0.0   
121958  0.000000  0.000000  0.000000  0.184918  0.183349  0.185237  0.0  0.0   

              09        10  ...   ok   

In [71]:
model.fit(X_train,y_train)

LogisticRegression(class_weight='balanced')

In [72]:
print(X_train.shape)
print(y_train.shape)

(799999, 144)
(799999,)


In [73]:
print(combined_features.shape)

(999999,)


In [74]:
prediction_on_training_data = model.predict(X_train)

accuracy_on_training_data = accuracy_score(y_train, prediction_on_training_data)

In [75]:
print(accuracy_on_training_data)

0.4998368747960935


In [80]:
# Example new patient data
new_patient_data = {
    'patient_age': 55,
    'smoking_status': 'Former Smoker',
    'cigarettes_per_day': 17,
    'has_other_cancer': 'Yes',
    'asthma_diagnosis': 'No',
    'liver_condition': 'Yes',
    'blood_pressure_status': 'High Blood Pressure',
    'diagnosis_date': '2018-11-03',
    'cancer_stage': '2',
    'treatment_start_date': '2018-11-21',
    'treatment_type': 'Surgery',
    'treatment_end_date': '2019-06-25'
}


# new_patient_data = {
#     'patient_age': 30,                  # younger age
#     'smoking_status': 'never smoked',   # no smoking
#     'cigarettes_per_day': 0,            # no cigarettes
#     'has_other_cancer': 'no',           # no other cancers
#     'asthma_diagnosis': 'no',           # no asthma
#     'liver_condition': 'no',            # no liver problems
#     'blood_pressure_status': 'normal',  # healthy BP
#     'diagnosis_date': '2024-01-01',
#     'cancer_stage': 'I',                # early-stage cancer
#     'treatment_start_date': '2024-01-15',
#     'treatment_type': 'surgery',        # effective early treatment
#     'treatment_end_date': '2024-02-15'
# }


# Combine into string
new_combined = (
    str(new_patient_data['patient_age']) + ' ' +
    str(new_patient_data['smoking_status']) + ' ' +
    str(new_patient_data['cigarettes_per_day']) + ' ' +
    str(new_patient_data['has_other_cancer']) + ' ' +
    str(new_patient_data['asthma_diagnosis']) + ' ' +
    str(new_patient_data['liver_condition']) + ' ' +
    str(new_patient_data['blood_pressure_status']) + ' ' +
    str(new_patient_data['diagnosis_date']) + ' ' +
    str(new_patient_data['cancer_stage']) + ' ' +
    str(new_patient_data['treatment_start_date']) + ' ' +
    str(new_patient_data['treatment_type']) + ' ' +
    str(new_patient_data['treatment_end_date'])
)

# Transform
new_vector = vectorizer.transform([new_combined])

# Predict
prediction = model.predict(new_vector)
proba = model.predict_proba(new_vector)

print("Predicted survival status:", prediction[0])
print("Probability of survival:", proba[0][1])
print("Probability of not surviving:", proba[0][0])


Predicted survival status: 1
Probability of survival: 0.5001185725495223
Probability of not surviving: 0.4998814274504777


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [77]:
from sklearn.metrics import classification_report
report = classification_report(y_train, prediction_on_training_data, target_names=['Not Survived (0)', 'Survived (1)'])
print(report)


                  precision    recall  f1-score   support

Not Survived (0)       0.78      0.50      0.61    624360
    Survived (1)       0.22      0.52      0.31    175639

        accuracy                           0.50    799999
       macro avg       0.50      0.51      0.46    799999
    weighted avg       0.66      0.50      0.54    799999



In [78]:
print(cancer_data['survival_status'].value_counts())


survival_status
0    780395
1    219604
Name: count, dtype: int64


In [83]:
import pandas as pd

# Load CSV
data = pd.read_csv('/content/test.csv')

# Combine features
combined_features = (
    data['patient_age'].astype(str) + ' ' +
    data['smoking_status'].astype(str) + ' ' +
    data['cigarettes_per_day'].astype(str) + ' ' +
    data['has_other_cancer'].astype(str) + ' ' +
    data['asthma_diagnosis'].astype(str) + ' ' +
    data['liver_condition'].astype(str) + ' ' +
    data['blood_pressure_status'].astype(str) + ' ' +
    data['diagnosis_date'].astype(str) + ' ' +
    data['cancer_stage'].astype(str) + ' ' +
    data['treatment_start_date'].astype(str) + ' ' +
    data['treatment_type'].astype(str) + ' ' +
    data['treatment_end_date'].astype(str)
)

# Vectorize
X_input = vectorizer.transform(combined_features)

# Predict
predictions = model.predict(X_input)

# Append predictions
data['predicted_survival_status'] = predictions

# Save to CSV
data.to_csv('/content/output_with_predictions.csv', index=False)
print("Done! Output saved as 'output_with_predictions.csv'")



from google.colab import files
files.download('/content/output_with_predictions.csv')


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


Done! Output saved as 'output_with_predictions.csv'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [89]:
# Select only necessary columns
output_df = data[['record_id', 'predicted_survival_status']].copy()

# Rename column
output_df.rename(columns={'predicted_survival_status': 'survival_status'}, inplace=True)

# Save
output_df.to_csv('/content/final_submission.csv', index=False)

# Download
from google.colab import files
files.download('/content/final_submission.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [84]:
import joblib


In [90]:
# Save the model to a .pkl file
joblib.dump(model, '/content/final_model.pkl')


['/content/final_model.pkl']

In [91]:
!ls -lh /content/final_model.pkl


-rw-r--r-- 1 root root 3.1K Jul 10 05:53 /content/final_model.pkl


In [92]:
from google.colab import files
files.download('/content/final_model.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>